In [2]:
#preprocess all masks
from pathlib import Path
import shutil
import numpy as np
from PIL import Image
import cv2
import datetime
n = 10
mask_dir = Path(f"/home/sashank/data/work/MTI/batches/batch_{n}/masks")
mask_proc_dir = Path(f"/home/sashank/data/work/MTI/batches/batch_{n}/masks_proc")
img_dir = Path(f"/home/sashank/data/work/MTI/batches/batch_{n}/rgb")
dataset_dir = Path("/home/sashank/data/work/datasets/003/train/")
rgb_dataset_dir = dataset_dir / "rgb"
mask_dataset_dir = dataset_dir / "masks"
if mask_proc_dir.exists():
    shutil.rmtree(mask_proc_dir)
mask_proc_dir.mkdir()
mask_files = [x for x in mask_dir.glob("*.png")]
print(len(mask_files))
for mask_file in mask_files:
    img_arr = cv2.imread(str(mask_file), cv2.IMREAD_GRAYSCALE) 
    img_arr = cv2.resize(img_arr, (1280, 720), interpolation=cv2.INTER_NEAREST)
    img_arr  = img_arr.copy()
    #Blur the image with a Gaussian filter
    img_arr = cv2.GaussianBlur(img_arr, (5, 5), 0)
    #Threshold the image, this implementation is absolute
    img_arr = cv2.threshold(img_arr, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
    img_arr = img_arr.astype(np.uint8)
    Image.fromarray(img_arr).save(mask_proc_dir / (mask_file.name.split(".")[0] + ".png"))

images = [x for x in img_dir.glob("*.jpg")]
added_img = 0
for image in images:
    mask_file = mask_proc_dir / ("mask_"+image.name.split(".")[0] + ".png")
    if mask_file.exists():
        added_img += 1
        shutil.copy(image, rgb_dataset_dir / image.name)
        shutil.copy(mask_file, mask_dataset_dir / mask_file.name)
    else:
        image.unlink()

readme_file = dataset_dir / "README.txt"
with open(readme_file, 'a') as f:
    f.write(f"Added {added_img} images from {mask_dir.parent.name} on {datetime.datetime.now()}\n")



200


In [1]:
from pathlib import Path
import pandas as pd
import shutil

labels_dir = Path("/home/sashank/Downloads/val_zed_labelled/")
labels_df = pd.read_csv(labels_dir / "gt.csv")
dataset_dir = Path("/home/sashank/Downloads/val_zed_classify_labels/")
if dataset_dir.exists():
    shutil.rmtree(dataset_dir)
dataset_dir.mkdir()
data_0 = dataset_dir / "0"
data_1 = dataset_dir / "1"
data_2 = dataset_dir / "2"
data_0.mkdir()
data_1.mkdir()
data_2.mkdir()
for i in [0, 1, 2]:
    data_i = dataset_dir / str(i)
    for index, row in labels_df.iterrows():
        if row["label"] == i:
            shutil.copy(labels_dir / row["image"], data_i / row["image"])

In [13]:
#clean up dataset by removing any images in rgb folder that have no corresponding mask
from pathlib import Path
import shutil
img_dir = Path("/home/sashank/data/work/MTI/batches/batch_3/rgb")
mask_proc_dir = Path("/home/sashank/data/work/MTI/batches/batch_3/masks_proc")
dataset_dir = Path("/home/sashank/data/work/datasets/003/train/")
rgb_dataset_dir = dataset_dir / "rgb"
mask_dataset_dir = dataset_dir / "masks"
images = [x for x in img_dir.glob("*.jpg")]
for image in images:
    mask_file = mask_proc_dir / ("mask_"+image.name.split(".")[0] + ".png")
    if mask_file.exists():
        shutil.copy(image, rgb_dataset_dir / image.name)
        shutil.copy(mask_file, mask_dataset_dir / mask_file.name)
    else:
        image.unlink()


